# News Agent

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama
ollama = ChatOllama(model="llama3.2:1b", temperature=0)    


## Creating tools

In [3]:
from typing import List, Dict, Optional
from urllib.parse import quote
import feedparser

class GoogleNews:

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]
    
    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:
        if not news_list:
            print("There is no news for the keyword.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_by_keyword(
            self, keyword: Optional[str] = None, k: int = 3
        ) -> List[Dict[str, str]]:
            
            if keyword:
                encoded_keyword = quote(keyword)
                url = f"https://news.google.com/rss/search?q={encoded_keyword}&hl=en&gl=US&ceid=US:en"
            else:
                url = f"https://news.google.com/rss?hl=en&gl=US&ceid=US:en"

            news_list = self._fetch_news(url, k)
            return self._collect_news(news_list)

In [4]:
from langchain.tools import tool
from typing import List, Dict

@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

print(f"Tool Name: {search_news.name}")
print(f"Tool Description: {search_news.description}")

Tool Name: search_news
Tool Description: Search Google News by input keyword


In [5]:
tools = [search_news]

## Generating Prompts for Agents

A prompt is text that describes the task the model will perform whose input is the tool name and its role.

- chat_history: A variable that stores previous conversation history (can be omitted if multi-turn support is not required).
- agent_scratchpad: A variable for temporary storage used by the agent.
- input: The user's input.

In [7]:
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are an helpfull assistant"
         "Make sure to use `search_news` tool for searching the news using keywords"),
         ("placeholder","{chat_history}"),
         ("human","{input}"),
         ("placeholder","{agent_scratchpad}")
    ]
)

In [10]:
# create agent
ollama_agent = create_tool_calling_agent(ollama,tools,prompt)

In [12]:
#  create Agent executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=ollama_agent,
    tools = tools,
    verbose=True,
    handle_parsing_errors=True
)
result = agent_executor.invoke({"input":"search news on Indian stocke market today"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'Indian stock market news today'}`


[{'url': 'https://news.google.com/rss/articles/CBMi2AFBVV95cUxQdjlaNG1ycnktcHdCUm9KNGZNMzBSX3M5MTVtSlpzaFlDQmhnZE1Eam5rZkhjVExOQ2FocWhHd2dhOVAwRFZ3ek9JRGFOWHpCTmhCV3luUloxVUp2YmhOLTV5MmNrMWdOa2dWdUJISFgyWkFmMFFQb05YRDUya0x1TkFqV1M1ZzlPWUNIdnZDT3o0ZEVHRlRvczN4V2gwOTEyZEY0Mk8tQWViTzQ3UGNucHE1Sm94cHE2UF8zVExweGU0WTBhQzAtaWtiVFp2S3VoY1U1QnRTZ23SAd4BQVVfeXFMTWJfWnI0VS0zaVJCMzBOM0E5SWlyVEhzLXhONDB1amE1MEdkVnExWjlpREZVOXhiUDZPVjJBckx3MDI4cDI1QjJKVUFXVXdYQTZzbkJRV0FHS0lzUW9WOFV0R1AzUmhkRFpZLW5Pd2Qtb0FnLWdBcEJTZzhvZUUxYk9LMnFEZWh5YXlZM3lOMEJKR19fS205V0lMWC1la2tVampWUHhURENYbjZieHU5VlY5Q3dDak1KOW5obE5VM2ZiTXJiQXllNEt3enVxSEQ0MXNiT0pwOTlxdFhwVmpR?oc=5', 'content': 'Stock Market Highlights: Nifty forms green candle, buy-on-dips favored above 22,725. How to trade on Thursday - The Economic Times'}, {'url': 'https://news.google.com/rss/articles/CBMixgFBVV95cUxPeTRiWk5SMmZwbjFyUmI5Wk1

In [13]:
result = agent_executor.invoke({"input":"sSearch the latest AI Investment news"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'latest AI investment news'}`


[{'url': 'https://news.google.com/rss/articles/CBMifkFVX3lxTE5GUXpWazJCelZMdE04MFlEeG1sNHFLWnhLcWJsWHYxQjRQeURNcnFod2dUUzM4TzYtVjEwVTNNRlZncXZ3U19PejRyZUtrNDhFU2ZJY2FrZlQyT2x6c0Nsci1vcjhpVnNQMUx6V204SXNzNkE0ZTNua2hvNGpydw?oc=5', 'content': "AI Stocks Face 'Show Me' Moment As China's DeepSeek Roils Capital Spending Outlook - Investor's Business Daily"}, {'url': 'https://news.google.com/rss/articles/CBMioAFBVV95cUxQMjRqcmFqa3FBaGxmdmZfQ0tPWFVFbnI0OThfeklwZ0FHOVZ4bG9fczhUUlBBbF82YXpfd0w4eVA3eXdqMndLTDNpYkV5THZkYVRmVXBJOFZjQlpfVmtJSWM0dUtQcWpnTVVZcFpZT3c4bW9GdHhwVndIZlhzRmRtTlQwY3AtQk1oMW5vMDUwOExORE1Ub040QXlVelhNak85?oc=5', 'content': 'Nvidia slashes stake in emerging rival as AI arms race heats up - TheStreet'}, {'url': 'https://news.google.com/rss/articles/CBMixgFBVV95cUxNTVZiaXNCY2JqdjJYTnZhZVF0R19hUUxOZjNNeE05dDdmOGQ5bUpKNGNMcVE4ZGFlTy1KY3lqSWVxcDhra1JRdmQzUDNBOG5RcHd4T0hBb

In [14]:
result = agent_executor.invoke({"input":"Search news about ranvir allahabadiya and create a post for instagram"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'ranvir allahabadiya'}`


[{'url': 'https://news.google.com/rss/articles/CBMi7gFBVV95cUxNSXN0WGZqOHhpY0lyU2xSbTdUYWhFOXF2RGVaa2hKdGZfcXBscEdJR2ZYU0t2MUtPeVNKUm05MnpKLW9qTVlhTDU5WU1VMWo4WE1pR2hmWlpzSTVRelZrdXpOU2hLUUhqaUVQLUdHU3ZuTGFDX1NYSTlHbEJtZ0xDVk5Ya1pDNG91elJJbDVsZC15YjZfT05qdDM3SHp4QkRjNGFwV0NPbm80QmdCVjMxZ2Q5ZWxPdjBXUkwteDk3b1dwdnFlWmpYY0tObV96Z2t3cDI1OGM5Sm1EV0swaXA5eGg1MnRZVEhoX1BaTW130gHzAUFVX3lxTE13UXVPYmZjZkdlc1g2clo4WVEweXpxUndGU01nWHhaN3g4SG5vb2twdHl6UWNNc2VwNnhHempfcUg5b2c1bXAydDV0R0pPbnBPZnE1UzNkMGpqcEJTRmZ3SmFZUXBoYlZic0E0SDh0ajZCckpaR0wyY2g2Q3g5bXIzS19FWXV0SEhkUWxKMzJrOXlCcTZxSzQwekRtbmxGSnd6dVpNOWN1SWxYQUdRSkdWZnA3bzZ3eXVSamNyLWE0YTdjQ2NxT0hXd1VUVUlCRTVTZnJUREs1WFRmQWVmRERyTmU0bXVqMm9EOFdkY3pQN0JRbw?oc=5', 'content': "Ranveer Allahbadia's BeerBiceps YouTube channel and all videos deleted after being hacked - The Economic Times"}, {'url': 'https://news.google.com/rss/articles/CBMi4wFBVV95